# This code file is to read jason data and create appropriate format

In [25]:
import json
import re
import pandas as pd
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
wnl = WordNetLemmatizer() 
sr = stopwords.words('english')

In [19]:
def count_word_char(line):
    '''
    For simplexity we can first ignore special character
    
    1. Split line if it is not a word character
    2. Iterated each word, count if it is one word, count total characters
    '''
    
    words = [word for word in re.split(r'\W', line) if word.isalnum()] # divide string based on symbol
    
    symbols = [word.strip() for word in re.split(r'\w', line) if word not in ['', ' ']] # devide string based on word, only keep symbol not '' or ' '
    
    wc, cc = 0, 0 # counting 
    word_dict = {} # word dictionary
    
    for word in words: # For each word in the word list
        
        cc += len(word) # add current word characters length
        
        if not word.isdecimal(): # If the word is a not a pure number
            wc += 1 # counter + 1
            
            # Save word frequency
            
            if word in word_dict: 
                word_dict[word] += 1
            else:
                word_dict[word] = 1  
    #print("""Symbols are: {}\nTotal words count: {}\nTotal character count: {}\n""".format(symbols, wc, cc))
    
    #print("-"*31)
    #print("{0:15}|{1:>15}".format("Word", "Frequency"))
    #print("-"*31)
    #for word, frequency in word_dict.items():
    #    print("{0:15}|{1:>15}".format(word, frequency))
    
    return symbols, wc, cc, word_dict

In [43]:
## read jason data 
file_name = 'data/review_train.json'
review_train = []
with open(file_name, 'r') as f:
    try:
        while True:
            line = f.readline()
            if line:
                r = json.loads(line)
                review_train.append(r)
            else:
                break
    except:
        f.close()

In [10]:
## one review example
review_example = review_train[1]
_, _, _, _ =count_word_char(review_example['text'])
tokens = count_word_char(review_example['text'])[3]

In [44]:
## 10 reviews try
testsize = 10
review_train_try = review_train[0:testsize]
# print(review_train_try)

[{'business_id': 31292, 'stars': 1.0, 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', 'date': '2013-05-07 04:34:36'}, {'business_id': 35344, 'stars': 5.0, 'text': "I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts th

In [45]:
for i in range(0,testsize):
        review = review_train_try[i]
        print(i)
        tokens = count_word_char(review['text'])[3]
        clean_tokens = dict()
        for token in tokens:
            ## change all words into lower form
            token_low = token.lower()
            ## lemmatization
            token_clear = wnl.lemmatize(token_low)
            if not token_clear in sr:
                clean_tokens.setdefault(token_clear, []).append(tokens[token])
        review_train_try[i]['text'] = clean_tokens

0
1
2
3
4
5
6
7
8
9


In [46]:
# print(review_train_try)

[{'business_id': 31292, 'stars': 1.0, 'text': {'total': [1], 'bill': [1], 'horrible': [1], 'service': [1], '8gs': [1], 'crook': [1], 'actually': [1], 'nerve': [1], 'charge': [1], 'u': [1], 'pill': [2], 'checked': [1], 'online': [1], 'cent': [1], 'avoid': [1], 'hospital': [1], 'er': [1], 'cost': [1]}, 'date': '2013-05-07 04:34:36'}, {'business_id': 35344, 'stars': 5.0, 'text': {'adore': [1], 'travis': [6], 'hard': [1], 'rock': [1], 'new': [2], 'kelly': [1], 'cardenas': [1], 'salon': [1], 'always': [1], 'fan': [1], 'great': [1], 'blowout': [5], 'stranger': [1], 'chain': [1], 'offer': [1], 'service': [1], 'however': [1], 'ha': [3], 'taken': [1], 'flawless': [1], 'whole': [1], 'level': [1], 'greets': [1], 'perfectly': [3], 'green': [1], 'swoosh': [1], 'otherwise': [1], 'styled': [1], 'black': [1], 'hair': [4], 'vega': [2], 'worthy': [1], 'rockstar': [1], 'outfit': [1], 'next': [2], 'come': [1], 'relaxing': [1], 'incredible': [1], 'shampoo': [2], 'get': [2, 1], 'full': [1], 'head': [1], 'me

In [47]:
review_train_try_df = pd.DataFrame(review_train_try)

In [49]:
print(review_train_try_df.head())

   business_id                 date  stars  \
0        31292  2013-05-07 04:34:36    1.0   
1        35344  2017-01-14 21:30:33    5.0   
2       152538  2016-11-09 20:09:03    5.0   
3        71871  2018-01-09 20:56:38    5.0   
4        64913  2018-01-30 23:07:38    1.0   

                                                text  
0  {'total': [1], 'bill': [1], 'horrible': [1], '...  
1  {'adore': [1], 'travis': [6], 'hard': [1], 'ro...  
2  {'say': [1], 'office': [4], 'really': [1], 'ha...  
3  {'went': [1], 'lunch': [1], 'steak': [1], 'san...  
4  {'today': [1, 2], 'wa': [16], 'second': [2, 1]...  
